In [ ]:
import numpy as np
import os
import sys
import warnings
import time

measure_sampling_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Measure_sampling')
trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Trees')

if measure_sampling_path not in sys.path:
    sys.path.append(measure_sampling_path)

if trees_path not in sys.path:
    sys.path.append(trees_path)

# Now import
from Gen_Path_and_AdaptedTrees import generate_adapted_tree
from Tree_Node import *
from TreeAnalysis import *
from TreeVisualization import *
from Save_Load_trees import *
from Tree_AWD_utilities import *

# Suppress sklearn warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Set parameters
num_paths = 100   # number of sample paths
time_steps = 4   # number of time steps per path
scale = 1
use_weights = 1

# Generate adapted trees for Brownian motion and financial model
print("Generating adapted tree from Brownian motion...")
bm_root, bm_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=scale, use_weights=use_weights, model='brownian', return_times=True)

print("Generating adapted tree from Financial Model...")
fin_root, fin_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=scale, use_weights=use_weights, model='financial', return_times=True)

print(f"Brownian Motion Adapted Tree generated successfully in {sum(bm_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {bm_time_tdetail[0]:.2f}s, Measure k-means: {bm_time_tdetail[1]:.2f}s, Tree construction: {bm_time_tdetail[2]:.2f}s")

print(f"Financial Model Adapted Tree generated successfully in {sum(fin_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {fin_time_tdetail[0]:.2f}s, Measure k-means: {fin_time_tdetail[1]:.2f}s, Tree construction: {fin_time_tdetail[2]:.2f}s")


Generating adapted tree from Brownian motion...
Generating adapted tree from Financial Model...
Brownian Motion Adapted Tree generated successfully in 0.09 seconds.
Time breakdown: Sample generation: 0.00s, Measure k-means: 0.09s, Tree construction: 0.00s
Financial Model Adapted Tree generated successfully in 0.06 seconds.
Time breakdown: Sample generation: 0.00s, Measure k-means: 0.06s, Tree construction: 0.00s


In [2]:
def print_results(distance, prob_matrix, elapsed_time):
    """Prints the nested distance, probability matrix, and computation time in a formatted way."""
    print(f"Nested Distance: {distance}\n")
    print(f"Final Probability Matrix:")
    for row in prob_matrix:
        print(" | ".join(f"{value:.5f}" for value in row))
    print(f"\nComputation Time: {elapsed_time:.4f} seconds\n")
    print("-" * 50 + "\n")

In [3]:
import sys
import os
import time

# Define the absolute path to the AWD_trees folder
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add it to sys.path if not already present
if awd_trees_path not in sys.path:
    sys.path.append(awd_trees_path)

import time
from Gurobi_AOT import *

formatted_tree_1 = get_sample_paths(bm_root)
formatted_tree_2 = get_sample_paths(fin_root)

def cost_function(x, y):
    """Cost function: L1 distance."""
    return np.sum(np.abs(x - y))

start_time = time.time()

val_gurobi = gurobi_bm(
    [formatted_tree_1, formatted_tree_2],
    f=cost_function,
    r_opti=0,
    causal=1,
    anticausal=1,
    outputflag=0
)

end_time = time.time()
elapsed_time_gurobi = end_time - start_time


print(val_gurobi, elapsed_time_gurobi)

Set parameter Username
Set parameter LicenseID to value 2604970
Academic license - for non-commercial use only - expires 2026-01-03
2.8161828899857655 2.3715929985046387


In [4]:
import time

# Define the absolute path to the AWD_trees folder
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add it to sys.path if not already present
if awd_trees_path not in sys.path:
    sys.path.append(awd_trees_path)


from Nested_Dist_Algo import compute_nested_distance

# Start timing the Sinkhorn with regularization computation


start_time = time.time()

max_depth = get_depth(bm_root)

start_time = time.time()

distance = compute_nested_distance(
    bm_root, fin_root, max_depth, method="solver_lp", return_matrix=False, lambda_reg=0
)

end_time = time.time()
elapsed_time = end_time - start_time

print(distance)
print(elapsed_time)

4.455352792668593
3.6105759143829346


In [5]:
# Define the absolute path to the AWD_trees folder
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add it to sys.path if not already present
if awd_trees_path not in sys.path:
    sys.path.append(awd_trees_path)


from Nested_Dist_Algo import compute_nested_distance
start_time = time.time()

max_depth = get_depth(bm_root)
distance = compute_nested_distance(
    bm_root, fin_root, max_depth, method="solver_pot", return_matrix=False, lambda_reg=0
)

end_time = time.time()
elapsed_time = end_time - start_time

print(distance)
print(elapsed_time)

4.455352792668593
0.6571199893951416


In [6]:
# Define the absolute path to the AWD_trees folder
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add it to sys.path if not already present
if awd_trees_path not in sys.path:
    sys.path.append(awd_trees_path)


from Nested_Dist_Algo import compute_nested_distance
start_time = time.time()

max_depth = get_depth(bm_root)

# Compute nested distance using Sinkhorn iteration with regularization
distance_sinkhorn_reg = compute_nested_distance(
    bm_root, fin_root, max_depth, method="Sinkhorn", return_matrix=False, lambda_reg=8
)

# End timing
end_time = time.time()
elapsed_time_sinkhorn_reg = end_time - start_time

print(distance_sinkhorn_reg)
print(elapsed_time_sinkhorn_reg)

4.512724653878329
3.2575552463531494


In [13]:
# Set parameters
num_paths = 200   # number of sample paths
time_steps = 5   # number of time steps per path
scale = 1
use_weights = 1


# Generate adapted trees for Brownian motion and financial model
print("Generating adapted tree from Brownian motion...")
bm_root, bm_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=1, use_weights=use_weights, model='brownian', return_times=True)

print("Generating adapted tree from Financial Model...")
fin_root, fin_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=0.5**2, use_weights=use_weights, model='brownian', return_times=True)

print(f"Brownian Motion Adapted Tree generated successfully in {sum(bm_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {bm_time_tdetail[0]:.2f}s, Measure k-means: {bm_time_tdetail[1]:.2f}s, Tree construction: {bm_time_tdetail[2]:.2f}s")

print(f"Financial Model Adapted Tree generated successfully in {sum(fin_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {fin_time_tdetail[0]:.2f}s, Measure k-means: {fin_time_tdetail[1]:.2f}s, Tree construction: {fin_time_tdetail[2]:.2f}s")

Generating adapted tree from Brownian motion...
Generating adapted tree from Financial Model...
Brownian Motion Adapted Tree generated successfully in 0.28 seconds.
Time breakdown: Sample generation: 0.00s, Measure k-means: 0.27s, Tree construction: 0.00s
Financial Model Adapted Tree generated successfully in 0.21 seconds.
Time breakdown: Sample generation: 0.00s, Measure k-means: 0.20s, Tree construction: 0.00s


In [14]:
start_time = time.time()

max_depth = get_depth(bm_root)
distance = compute_nested_distance(
    bm_root, fin_root, max_depth, method="solver_pot", return_matrix=False, lambda_reg=0
)

end_time = time.time()
elapsed_time = end_time - start_time

print(distance)
print(elapsed_time)

5.5568406512517825
9.633142232894897
